# Air Traffic Control For Emails
#### _by **Raghav Gurung**_

In [1]:
# Importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


---
## Reading Email Report

The Email Report has one year (Week 1 - Week 52) history of all the campaigns that were deployed by the company. The report includes each campaign’s performance details - open rate, click rate, unique click rate, unsubscribe rate 3,372 records with the following columns:

- **Week** - The week in which the email was deployed
- **Business** - The business (Clothing, Grocery and Homeware) for which the email was done
- **Message** - The unique identifier for each campaign
- **Email Type** - The purpose of the email i.e. some of the campaigns are done by the company to inform the customers about something like closing/opening of store, offers etc. where as the others are done for trade
- **Delivered** - The number of customers who received the email
- **Opened** - The number of customers who opened the email
- **Clicked** - The number of customers who clicked on any section in the email
- **Unsubscribed** - The number of customers who clicked on the unsubscribe link in the email

In [2]:
email_report = pd.read_csv('Email_Report.csv')
email_report.head()

,Week,Business,Message,Email_Type,Delivered,Opened,Clicked,Unsubscribed
0,1,Clothing,ACQ_BABYKIDS_15_01_06_381,Trade,294890,73205,10249,1115
1,1,Clothing,ACQ_INSTORE_15_01_06_382,Trade,308032,82619,18090,1119
2,1,Homeware,ABASKET_15_01_01_902,Trade,1405,747,112,0
3,1,Homeware,ABASKET_15_01_02_902,Trade,1314,690,135,2
4,1,Homeware,ABASKET_15_01_05_902,Trade,1324,725,153,1


## Exploring Data
- Check for the correctness of the file
- Check if we have all the required columns and if they are in the correct format


---
## Data Cleaning
Check if there are any discrepancies in the data. For Example: For every email, the number of customers who opened, clicked or unsubscribed should always be less than the customers who received the email. Let’s check if that is true.

In [3]:
print(email_report[email_report.Opened > email_report.Delivered].shape[0])
print(email_report[email_report.Clicked > email_report.Delivered].shape[0])
print(email_report[email_report.Unsubscribed > email_report.Opened].shape[0])
print(email_report[email_report.Unsubscribed > email_report.Clicked].shape[0])

5
5
146
283


In [4]:
# Remove the data issues
email_report_cleaned = email_report[email_report.Opened < email_report.Delivered]
email_report_cleaned = email_report[email_report.Clicked < email_report.Delivered]
email_report_cleaned = email_report[email_report.Unsubscribed < email_report.Opened]
email_report_cleaned = email_report[email_report.Unsubscribed < email_report.Clicked]
print(email_report_cleaned.shape)

(3070, 8)


In [5]:
# Adding email engagement metrics
email_report_cleaned['Open_Rate'] = email_report_cleaned.Opened/email_report_cleaned.Delivered
email_report_cleaned['Click_Rate'] = email_report_cleaned.Clicked/email_report_cleaned.Delivered
email_report_cleaned['Unsub_Rate'] = email_report_cleaned.Unsubscribed/email_report_cleaned.Delivered

In [6]:
email_report_cleaned.head()

,Week,Business,Message,Email_Type,Delivered,Opened,Clicked,Unsubscribed,Open_Rate,Click_Rate,Unsub_Rate
0,1,Clothing,ACQ_BABYKIDS_15_01_06_381,Trade,294890,73205,10249,1115,0.248245,0.034755,0.003781
1,1,Clothing,ACQ_INSTORE_15_01_06_382,Trade,308032,82619,18090,1119,0.268216,0.058728,0.003633
2,1,Homeware,ABASKET_15_01_01_902,Trade,1405,747,112,0,0.531673,0.079715,0.000000
3,1,Homeware,ABASKET_15_01_02_902,Trade,1314,690,135,2,0.525114,0.102740,0.001522
4,1,Homeware,ABASKET_15_01_05_902,Trade,1324,725,153,1,0.547583,0.115559,0.000755


---
## Model Building
- The first step will be to divide the campaigns into 4 different groups based on their open, click and unsubscribe rate
- We will not take click rate into consideration for Inform type of campaigns as the agenda was to just make the customer open that email and not click
- Email Score will be calculated at a business and email type level
- Finally, we will normailize the email score so as to decide the priority of the campaigns on a given day

In [7]:
train = email_report_cleaned.copy(deep=True)
train.head()

,Week,Business,Message,Email_Type,Delivered,Opened,Clicked,Unsubscribed,Open_Rate,Click_Rate,Unsub_Rate
0,1,Clothing,ACQ_BABYKIDS_15_01_06_381,Trade,294890,73205,10249,1115,0.248245,0.034755,0.003781
1,1,Clothing,ACQ_INSTORE_15_01_06_382,Trade,308032,82619,18090,1119,0.268216,0.058728,0.003633
2,1,Homeware,ABASKET_15_01_01_902,Trade,1405,747,112,0,0.531673,0.079715,0.000000
3,1,Homeware,ABASKET_15_01_02_902,Trade,1314,690,135,2,0.525114,0.102740,0.001522
4,1,Homeware,ABASKET_15_01_05_902,Trade,1324,725,153,1,0.547583,0.115559,0.000755


In [8]:
summary = pd.crosstab(index=train.Business,
                      columns=train.Email_Type)
summary = summary.apply(lambda x: x / summary.sum(axis=1)) * 100
summary

Email_Type,Inform,Trade
Business,,
Clothing,7.702350,92.297650
Grocery,22.904564,77.095436
Homeware,12.192903,87.807097


## Open Score
The campaigns will be divided into 4 groups based on their open rates

In [9]:
open_rate_summary = train.groupby(['Business', 'Email_Type'])['Open_Rate'].describe()

In [10]:
def create_open_score_metric(row):
    global open_rate_summary
    if row.Open_Rate > open_rate_summary.loc[row.Business, row.Email_Type]['min']\
      and row.Open_Rate < open_rate_summary.loc[row.Business, row.Email_Type]['25%']:
        return 1
    elif row.Open_Rate > open_rate_summary.loc[row.Business, row.Email_Type]['25%']\
      and row.Open_Rate < open_rate_summary.loc[row.Business, row.Email_Type]['50%']:
        return 2
    elif row.Open_Rate > open_rate_summary.loc[row.Business, row.Email_Type]['50%']\
      and row.Open_Rate < open_rate_summary.loc[row.Business, row.Email_Type]['75%']:
        return 3
    elif row.Open_Rate > open_rate_summary.loc[row.Business, row.Email_Type]['75%']\
      and row.Open_Rate < open_rate_summary.loc[row.Business, row.Email_Type]['max']:
        return 4
    else:
        return 0

train['Open_Score'] = train.apply(create_open_score_metric, axis=1)

---
## Click Score
The campaigns will be divided into 4 groups based on their click rates. However, we will ignore the Inform type campaigns as we don’t want the customers to click on such campaigns

In [11]:
click_rate_summary = train.groupby(['Business', 'Email_Type'])['Click_Rate'].describe()

def create_click_score_metric(row):
    if row.Email_Type == 'Inform':
        return 0
    else:
        if row.Click_Rate > click_rate_summary.loc[row.Business, row.Email_Type]['min']\
          and row.Click_Rate < click_rate_summary.loc[row.Business, row.Email_Type]['25%']:
            return 1
        elif row.Click_Rate > click_rate_summary.loc[row.Business, row.Email_Type]['25%']\
          and row.Click_Rate < click_rate_summary.loc[row.Business, row.Email_Type]['50%']:
            return 2
        elif row.Click_Rate > click_rate_summary.loc[row.Business, row.Email_Type]['50%']\
          and row.Click_Rate < click_rate_summary.loc[row.Business, row.Email_Type]['75%']:
            return 3
        elif row.Click_Rate > click_rate_summary.loc[row.Business, row.Email_Type]['75%']\
          and row.Click_Rate < click_rate_summary.loc[row.Business, row.Email_Type]['max']:
            return 4
        else:
            return 0
    
train['Click_Score'] = train.apply(create_click_score_metric, axis=1)

## Unsubscribe Score
Ideally, we would want our unsubscribe rate to be less therefore the campaigns with high unsubscribe rate will be penalized by assigning a high negative score. Also, in this case we will not calculate it at a business or an email type because our aim is to keep our customers engaged in every communication and not to lose them by sending irrelevant communications

In [12]:
train.Unsub_Rate.describe()

count    3070.000000
mean        0.002973
std         0.015970
min         0.000000
25%         0.000000
50%         0.000072
75%         0.000989
max         0.504425
Name: Unsub_Rate, dtype: float64

In [13]:
# Unsubcribe Score
def create_unsub_rate_metric(row):
    if row > 0.0010 and row <= 0.0030:
        return -2
    elif row > 0.0030 and row <= 0.0050:
        return -3
    elif row > 0.0050:
        return -4
    else:
        return 0

train['Unsub_Score'] = train['Unsub_Rate'].apply(create_unsub_rate_metric)


---
## Email Score
The Email Score will be calculated by incorporating the scores calculated above

- The first step will be to assign weights to each of the given components depending on the email type
- The second step will involve combining these individual scores into one final score
- Calculate a final normalize score that will be used to priortize the campaigns on a given day

In [14]:
# Email Score
# ES for Inform = 5 x Open Score + Unsubscribe Score                
# ES for Trade =    2 x Open Score + 3 x Click Score + Unsubscribe Score   

def create_email_score_metric(row):
    if row.Email_Type == 'Inform':
        return 5 * row.Open_Score + row.Unsub_Score
    else:
        return 2 * row.Open_Score + 3 * row.Click_Score + row.Unsub_Score
    
train['Email_Score'] = train.apply(create_email_score_metric, axis=1)
train['Norm_Email_Score'] = train['Email_Score'].apply(lambda x: x / (max(train['Email_Score'] - min(train['Email_Score']))))


## Model Application
### Campaign Priortization

Let’s say on a given day, we have 4 campaigns - A, B, C and D with Email Scores as 0.2, 0.4, 0.7 and 0.9 respectively. TO make sure that a customer receive only a single email in a day, we can rank these campaigns as per their ES like the followuing:

1. D
2. C
3. B
4. A

Now, say there is a customer “Pankaj” who is a part of the targeting for campaigns D, C and B. By this scoring, we will only send him campaign D and suppress him from receving B and C. This way, the customer receives our best campaign.

**Campaign Optimization**

This scoring can also be used to stop the campaigns which are not performing well. For Example: A trade campaign in Grocery was launched and after 6 deployments its final average score is 0.4. Looking at the results below, we can clearly say that this campaign is not performing well (as its score is below 50 percentile) and therefore can be stopped. Thus, it can help in optimizing a firm’s marketing strategy.



In [15]:
train.groupby(['Business', 'Email_Type'])['Norm_Email_Score'].describe().reset_index()

,Business,Email_Type,count,mean,std,min,25%,50%,75%,max
0,Clothing,Inform,59.0,0.494350,0.256321,0.000000,0.208333,0.416667,0.625000,0.833333
1,Clothing,Trade,707.0,0.506424,0.228714,0.041667,0.291667,0.500000,0.708333,0.833333
2,Grocery,Inform,276.0,0.446256,0.254887,-0.166667,0.208333,0.416667,0.625000,0.833333
3,Grocery,Trade,929.0,0.473134,0.217561,0.000000,0.291667,0.500000,0.666667,0.833333
4,Homeware,Inform,134.0,0.478234,0.243049,0.000000,0.208333,0.416667,0.625000,0.833333
5,Homeware,Trade,965.0,0.499611,0.211784,-0.083333,0.333333,0.500000,0.666667,0.833333


**Customer Segmentation**

A similar score can be calculated at a customer level based on the number of emails he/she has opened/clicked and thus the customers with high score can be targeted frequently as compared to the other customers. We can also also include the revenue metric while calculating the final score.

Lets understand by importing a sample customer file with 1306 unique customers. This file has: * Customer Number: Customer Identifier * Email_Address: Email of the customer * Business: from which business he received an email * Received: Total number of Emails Received * Opened: Number of emails opened * Clicked: Number of emails clicked * Open Rate: Opened/Received * Click Rate: Clicked/Received

Here, we are not taking the unsubscribe, because once a customer unsubscribes, he will never receive an email again so by default the value will be either 1 or 0

In [16]:
customer = pd.read_csv('Customer_Report.csv')
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1418 entries, 0 to 1417
Data columns (total 8 columns):
Customer_Number    1418 non-null int64
email_address      1418 non-null object
Business           1418 non-null object
Received           1418 non-null int64
Opened             1418 non-null int64
Clicked            1418 non-null int64
Open Rate          1418 non-null float64
Click Rate         1418 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 88.8+ KB


In [17]:
customer.head()

,Customer_Number,email_address,Business,Received,Opened,Clicked,Open Rate,Click Rate
0,221931897,awat@wp.pl,Homeware,3,1,0,0.33,0.00
1,221934416,lesl@talktalk.net,Grocery,4,1,0,0.25,0.00
2,221935037,andy@live.co.uk,Homeware,3,1,0,0.33,0.00
3,221935420,b001@rocketmail.com,Homeware,4,1,0,0.25,0.00
4,221935568,bill@live.co.uk,Homeware,7,3,2,0.43,0.29


In [18]:
train.groupby(['Business'])['Open_Rate'].median()

Business
Clothing    0.406639
Grocery     0.371651
Homeware    0.426606
Name: Open_Rate, dtype: float64

In [19]:
customer.groupby(['Business'])['Open Rate'].median()

Business
Clothing    0.33
Grocery     0.38
Homeware    0.40
Name: Open Rate, dtype: float64

---
##### We can see that for Clothing, Grocery and Homeware, the median open rate is 40.64%, 37.12% and 42.65% respectively. Now, let’s count the number of customers whose engagement is less than the threshold.

In [20]:
customer[(customer.Business=='Clothing') & (customer['Open Rate'] > 0.4064)].shape[0] \
 / customer[customer.Business=='Clothing'].shape[0]

0.1864406779661017

In [21]:
customer[(customer.Business=='Grocery') & (customer['Open Rate'] > 0.3712)].shape[0] \
 / customer[customer.Business=='Grocery'].shape[0]

0.5418994413407822

In [22]:
customer[(customer.Business=='Homeware') & (customer['Open Rate'] > 0.4265)].shape[0] \
 / customer[customer.Business=='Homeware'].shape[0]

0.3150684931506849


   - We can clearly see that only 18.6% of the customers who receive Clothing emails are actually engaged with the content
   - 54% of the customers who received Grocery emails are actually interested in them
   - 31.5% of the customers are interested in receiving Homeware emails

Hence, we can stop sending the emails to Zombies (basically the non-engaged customers)

---
---